In [ ]:
# get new infections deaths 30 days
# groupby country new smoothed days (past 30 days)

In [73]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

In [26]:
file = 'owid-covid-data-large.csv'
target = 'total_deaths_per_million'
threshhold = 2
node_scalar = 4
epochs = 5000

In [115]:

np.set_printoptions(precision=3, suppress=True)



def clean_data(raw_dataset, drop=True, impute=True):

    dataset = raw_dataset.copy()
    #dataset = dataset.iloc[1:, :]
    dataset.isna().sum()
    #dataset = dataset.dropna(axis=1, how='all')
    #dataset = dataset.dropna(axis=0, thresh=19)
    drop_horizontally = [
        'people_vaccinated_per_hundred',
        'total_boosters_per_hundred',
        'stringency_index',
        'population_density',
        'median_age', # maybe consider imputing 
        'gdp_per_capita',
        'cardiovasc_death_rate',
        'diabetes_prevalence',
        'female_smokers', #may need to remove missing 34.2%
        'male_smokers', # same reasoning ^
        'hospital_beds_per_thousand',
        'life_expectancy',
        'human_development_index'
        ]
    
    
    data_to_impute_logically = [
        'total_boosters_per_hundred'
    ]
    dataset = dataset.drop(
        ['iso_code',
         #'location',
         #'continent',
         #'date',
         'total_cases',
         'new_cases',
         'new_cases_smoothed',
         'total_deaths',
         'new_deaths',
         'new_deaths_smoothed',
         'total_cases_per_million',
         'new_cases_smoothed_per_million',
         #'total_deaths_per_million',
         'new_deaths_per_million',
         'new_deaths_smoothed_per_million',
         'reproduction_rate',
         'icu_patients',
         'icu_patients_per_million',
         'weekly_icu_admissions',
         'weekly_icu_admissions_per_million',
         'hosp_patients',
         'hosp_patients_per_million',
         'weekly_hosp_admissions',
         'weekly_hosp_admissions_per_million',
         'total_tests',
         'new_tests',
         'new_tests_smoothed',
         'total_tests_per_thousand',
         'new_tests_per_thousand',
         'new_tests_smoothed',
         'new_tests_smoothed_per_thousand',
         'positive_rate',
         'tests_per_case',
         'tests_units',
         'total_vaccinations',
         'people_vaccinated',
         'people_fully_vaccinated',
         'total_boosters',
         'new_vaccinations',
         #'new_vaccinations_smoothed',
         'total_vaccinations_per_hundred',
         #'people_vaccinated_per_hundred',
         'people_fully_vaccinated_per_hundred',
         'new_vaccinations_smoothed_per_million',
         'new_people_vaccinated_smoothed',
         'new_people_vaccinated_smoothed_per_hundred',
         'population',
         'extreme_poverty',
         'handwashing_facilities', #might add back, high correlation
         'excess_mortality_cumulative_absolute',
         'excess_mortality_cumulative',
         'excess_mortality',
         'excess_mortality_cumulative_per_million',
         'aged_70_older',
        ], axis=1)
    print(len(dataset.columns))
    dataset = pd.get_dummies(
        dataset, columns=['continent'], prefix='', prefix_sep='')
    if drop:
        dataset = dataset.dropna(subset=[target])
        dataset = dataset.dropna(subset=drop_horizontally)
        #dataset = dataset.dropna(thresh=dataset.shape[1]-threshhold, axis=0)
        print(dataset.loc[:, dataset.isna().any()])
    #if impute:
    #    impute_columns = dataset.columns
    #    for column in impute_columns:
    #        dataset[column] = dataset[column].fillna(dataset[column].mean())
    return (dataset)

In [116]:
raw_dataset = pd.read_csv(file, na_values='?', comment='\t', sep=',')
dataset = clean_data(raw_dataset)
features = len(dataset.index)

20
        new_cases_per_million  new_vaccinations_smoothed
2552                  200.020                     5080.0
2555                    0.000                     5739.0
2556                  396.187                     5709.0
2558                  269.730                     6447.0
2559                  241.706                     6481.0
...                       ...                        ...
223292                  0.125                     2889.0
223293                  2.126                     4424.0
223295                  0.000                     5140.0
223297                  0.000                     4037.0
223298                  3.251                     3870.0

[20753 rows x 2 columns]


In [119]:
print(f'Features: {features}')

Features: 20753


In [151]:
current_date_df = dataset[dataset['date'].iloc[-100] == dataset['date']]

In [159]:
last_thirty_df = dataset[dataset['date'] > '2022-09-01']

In [118]:
# find 90th percentile of Stringency, Population Density, Median Age, 65 oldder, gdp, male smoker, 
# hospital beds, life expenctancy, hdi
attributes = [
    'new_vaccinations_smoothed', 'people_vaccinated_per_hundred',
    'total_boosters_per_hundred', 'stringency_index', 'population_density',
    'median_age', 'aged_65_older', 'gdp_per_capita', 'male_smokers',
    'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 
    'new_cases_per_million'
]

                               90th Percentile
new_vaccinations_smoothed          145990.2000
people_vaccinated_per_hundred          90.7300
total_boosters_per_hundred             80.4490
stringency_index                       37.0400
population_density                    547.9452
median_age                             45.3000
aged_65_older                          20.3960
gdp_per_capita                      57410.1660
male_smokers                           41.5000
hospital_beds_per_thousand              8.0000
life_expectancy                        83.5100
human_development_index                 0.9400
new_cases_per_million                 630.4004


In [176]:
country = 'Thailand'

In [163]:
list(dataset['location'].unique())

['Albania',
 'Algeria',
 'Argentina',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Cambodia',
 'Canada',
 'Cape Verde',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gambia',
 'Germany',
 'Ghana',
 'Greece',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Kazakhstan',
 'Kenya',
 'Kiribati',
 'Kuwait',
 'Latvia',
 'Lebanon',
 'Liberia',
 'Lithuania',
 'Luxembourg',
 'Malawi',
 'Malaysia',
 'Malta',
 'Mauritius',
 'Mexico',
 'Moldova',
 'Mongolia',
 'Morocco',
 'Mozambique',
 'Myanmar',
 'Nepal',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Oman',
 'Pakistan',
 'Panama',
 'Paraguay',
 'Philippines',
 'Poland',
 'Portugal',
 'Qatar',
 'Russia',
 'S

In [170]:
print(current_date_df.loc[current_date_df['location'] == country].groupby('location').mean())

          new_cases_per_million  total_deaths_per_million  \
location                                                    
Zimbabwe                 11.755                   342.764   

          new_vaccinations_smoothed  people_vaccinated_per_hundred  \
location                                                             
Zimbabwe                    74788.0                           37.7   

          total_boosters_per_hundred  stringency_index  population_density  \
location                                                                     
Zimbabwe                        4.51              47.2              42.729   

          median_age  aged_65_older  gdp_per_capita  ...  male_smokers  \
location                                             ...                 
Zimbabwe        19.6          2.822        1899.775  ...          30.7   

          hospital_beds_per_thousand  life_expectancy  \
location                                                
Zimbabwe                         

/tmp/ipykernel_70446/1066781161.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(current_date_df.loc[current_date_df['location'] == country].groupby('location').mean())


In [194]:
last_thirty_df.loc[last_thirty_df['location'] == country].groupby('location').mean()

/tmp/ipykernel_70446/3408023322.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  last_thirty_df.loc[last_thirty_df['location'] == country].groupby('location').mean()


,new_cases_per_million,total_deaths_per_million,new_vaccinations_smoothed,people_vaccinated_per_hundred,total_boosters_per_hundred,stringency_index,population_density,median_age,aged_65_older,gdp_per_capita,...,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,Africa,Asia,Europe,North America,Oceania,South America
location,,,,,,,,,,,,,,,,,,,,,
Thailand,13.081517,455.365759,20304.0,79.576897,44.597241,29.63,135.132,40.1,11.373,16277.671,...,38.8,2.1,77.15,0.777,0.0,1.0,0.0,0.0,0.0,0.0


In [179]:
equal_weighted_country_df = last_thirty_df.groupby('location').mean()

/tmp/ipykernel_70446/1788821384.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  equal_weighted_country_df = last_thirty_df.groupby('location').mean()


In [180]:
d = {}
for attr in attributes:
    percentile = equal_weighted_country_df[attr].quantile(0.90) # 90th percentile
    d[attr] = percentile
percentile_df = pd.DataFrame.from_dict(d, orient='index', columns=['90th Percentile'])
print(percentile_df)

                               90th Percentile
new_vaccinations_smoothed        122664.000000
people_vaccinated_per_hundred        91.172917
total_boosters_per_hundred           72.946094
stringency_index                     39.298000
population_density                  445.637700
median_age                           43.400000
aged_65_older                        19.435000
gdp_per_capita                    48293.542000
male_smokers                         47.000000
hospital_beds_per_thousand            6.614000
life_expectancy                      82.647000
human_development_index               0.931900
new_cases_per_million               379.248103


In [193]:
#json, 'people_vaccinated_per_hundred',
#    'total_boosters_per_hundred', 'stringency_index', 'population_density',
#    'median_age', 'aged_65_older', 'gdp_per_capita', 'male_smokers',
#    'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 

equal_weighted_country_df.to_json(orient='index')

'{"Albania":{"new_cases_per_million":30.476,"total_deaths_per_million":1255.6445,"new_vaccinations_smoothed":644.5,"people_vaccinated_per_hundred":46.81,"total_boosters_per_hundred":12.41,"stringency_index":11.11,"population_density":104.871,"median_age":38.0,"aged_65_older":13.188,"gdp_per_capita":11803.431,"cardiovasc_death_rate":304.195,"diabetes_prevalence":10.08,"female_smokers":7.1,"male_smokers":51.2,"hospital_beds_per_thousand":2.89,"life_expectancy":78.57,"human_development_index":0.795,"Africa":0.0,"Asia":0.0,"Europe":1.0,"North America":0.0,"Oceania":0.0,"South America":0.0},"Algeria":{"new_cases_per_million":0.385,"total_deaths_per_million":155.711,"new_vaccinations_smoothed":628.0,"people_vaccinated_per_hundred":17.75,"total_boosters_per_hundred":1.3,"stringency_index":45.32,"population_density":17.348,"median_age":29.1,"aged_65_older":6.211,"gdp_per_capita":13913.839,"cardiovasc_death_rate":278.364,"diabetes_prevalence":6.73,"female_smokers":0.7,"male_smokers":30.4,"hospi

In [196]:
percentile_df.to_json(orient='index')

'{"new_vaccinations_smoothed":{"90th Percentile":122664.0},"people_vaccinated_per_hundred":{"90th Percentile":91.1729174194},"total_boosters_per_hundred":{"90th Percentile":72.94609375},"stringency_index":{"90th Percentile":39.298},"population_density":{"90th Percentile":445.6377},"median_age":{"90th Percentile":43.4},"aged_65_older":{"90th Percentile":19.435},"gdp_per_capita":{"90th Percentile":48293.542},"male_smokers":{"90th Percentile":47.0},"hospital_beds_per_thousand":{"90th Percentile":6.614},"life_expectancy":{"90th Percentile":82.647},"human_development_index":{"90th Percentile":0.9319},"new_cases_per_million":{"90th Percentile":379.248102621}}'